<a href="https://colab.research.google.com/github/dhanusharer/DL-practise/blob/main/keras03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10354527928010911829
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14426112000
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11254655553476784759
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

# ***create CNN model and optimize it using keras***

In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
print(tf.__version__)

2.19.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):
  model=keras.Sequential([
  keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
      activation='relu',
      input_shape=(28,28,1)
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(
      units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
      activation='relu'
  ),
  keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [13]:
from kerastuner import RandomSearch # Corrected capitalization
# from kerastuner.engine.Hyperparameters # Removed incomplete import
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion") # Corrected string and parameters

/tmp/ipython-input-2931396316.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch # Corrected capitalization
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.8701666593551636

Best val_accuracy So Far: 0.9049999713897705
Total elapsed time: 00h 02m 05s
